In [ ]:
from ultralytics import YOLO

In [ ]:
weights_path = 'runs/detect/train3/weights/best.pt'

In [ ]:
model = YOLO(weights_path)

In [ ]:
from PIL import Image, ImageDraw

In [ ]:
i = Image.open('test/1.png')
i = i.convert('RGB')
i

In [ ]:
pi_img_width, pi_img_height = 1920, 1080

crop_window = 450, 350, 1440, 1080
orig_size = crop_window[2] - crop_window[0], crop_window[3] - crop_window[1]
img = i.crop(crop_window)
img

In [ ]:
from torchvision.transforms import v2

transforms = v2.Compose([
    v2.Resize((640, 640)),
    v2.PILToTensor()
])

In [ ]:
import torch

t = transforms(img)

t = t.to(torch.float32)
t /= 255.0
t = t.unsqueeze(0)
t.shape

In [ ]:
preds = model(t)

In [ ]:
len(preds)

In [ ]:
boxes = preds[0].boxes.numpy()
boxes

In [ ]:
i2 = img.copy()
draw = ImageDraw.Draw(i2)

def denorm(x0, y0, x1, y1, orig_size=orig_size, new_size=(640, 640)):
    orig_w, orig_h = orig_size
    new_w, new_h = new_size
    
    cw, ch = orig_w / new_w, orig_h / new_h
    
    x0 = x0 * new_w * cw
    x1 = x1 * new_w * cw

    y0 = y0 * new_h * ch
    y1 = y1 * new_h * ch
    
    return x0, y0, x1, y1

for i, box in enumerate(boxes.xyxyn):
    cls = boxes.cls[i]
    conf = boxes.conf[i]
    
    xmin, ymin, xmax, ymax = box
    print(xmin, ymin, xmax, ymax)
    xmin, ymin, xmax, ymax = denorm(xmin, ymin, xmax, ymax)

    draw.rectangle(((xmin, ymin), (xmax, ymax)), outline='red', width=5)
    draw.text((xmin+7, ymin+10), f'p={conf}', fill='red')
    draw.text((xmin+7, ymin-10), f'c={cls}', fill='green')
    
i2

In [ ]:
i2.save('out.png')